In [31]:
import tensorflow;
import pandas;
from sklearn.model_selection import train_test_split;
from tensorflow import keras;
from tensorflow.keras.models import Sequential;
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt;
from sklearn.decomposition import PCA;
from tensorflow.keras.utils import plot_model;
from tensorflow.keras.regularizers import l2;

## Data Import

In [32]:
# Loading data from pre-cleaned csv file
dataframe = pandas.read_csv("../credit_card_clients.csv");

## Feature Correlation

In this section I set the treshold with 0.2. Any feature below 0.2 will be discarded.

In [33]:
# Correlation Heatmap
correlation = dataframe.corr();

TRESHOLD = 0;
print("Relationship : Correlation Score");

columns_to_be_dropped = [];
for i in correlation.columns:
    correlation_score = correlation[i]["LABEL"];
    print(i, "-> LABEL : ", correlation_score);

    if(correlation_score < TRESHOLD):
        columns_to_be_dropped.append(i);
        print("Dropping ", i);

dataframe = dataframe.drop(columns=columns_to_be_dropped);

Relationship : Correlation Score
ID -> LABEL :  -0.013951954838986251
Dropping  ID
LIMIT_BAL -> LABEL :  -0.1535198763935072
Dropping  LIMIT_BAL
SEX -> LABEL :  -0.03996057770544172
Dropping  SEX
EDUCATION -> LABEL :  0.028006077656250204
MARRIAGE -> LABEL :  -0.024339215683404438
Dropping  MARRIAGE
AGE -> LABEL :  0.013889834301962887
PAY_0 -> LABEL :  0.32479372847862237
PAY_2 -> LABEL :  0.2635512016721678
PAY_3 -> LABEL :  0.23525251372491712
PAY_4 -> LABEL :  0.21661363684242388
PAY_5 -> LABEL :  0.2041489138761645
PAY_6 -> LABEL :  0.18686636165354611
BILL_AMT1 -> LABEL :  -0.019644197143221562
Dropping  BILL_AMT1
BILL_AMT2 -> LABEL :  -0.014193218088215756
Dropping  BILL_AMT2
BILL_AMT3 -> LABEL :  -0.014075518043214726
Dropping  BILL_AMT3
BILL_AMT4 -> LABEL :  -0.010156495880289674
Dropping  BILL_AMT4
BILL_AMT5 -> LABEL :  -0.006760463841014779
Dropping  BILL_AMT5
BILL_AMT6 -> LABEL :  -0.005372314914815558
Dropping  BILL_AMT6
PAY_AMT1 -> LABEL :  -0.07292948777785163
Dropping  

## Data Pre-Processing
Checkout the `main.ipynb` since I just copy-pasting the whole thing from it.

In [34]:
# Some education data contains 0, 5, and 6. I eliminating "em.

invalid_education_data = [];
for idx, e in enumerate(dataframe['EDUCATION']):
    if(e > 4 or e < 1):
        invalid_education_data.append(idx);

dataframe = dataframe.drop(invalid_education_data);

# Make one hot encoding for Educuation and PAY_0 to 6 since the data is an ordinal data
hot_encoded_education = pandas.get_dummies(dataframe['EDUCATION'], prefix = "education");
hot_encoded_pay_0 = pandas.get_dummies(dataframe['PAY_0'], prefix = "pay_0");
# hot_encoded_pay_1 = pandas.get_dummies(dataframe['PAY_1'], prefix = "pay_1");
hot_encoded_pay_2 = pandas.get_dummies(dataframe['PAY_2'], prefix = "pay_2");
hot_encoded_pay_3 = pandas.get_dummies(dataframe['PAY_3'], prefix = "pay_3");
hot_encoded_pay_4 = pandas.get_dummies(dataframe['PAY_4'], prefix = "pay_4");
hot_encoded_pay_5 = pandas.get_dummies(dataframe['PAY_5'], prefix = "pay_5");
hot_encoded_pay_6 = pandas.get_dummies(dataframe['PAY_6'], prefix = "pay_6");

# Merge the hot_encoded with the main dataframe
for i in [hot_encoded_education, hot_encoded_pay_0, hot_encoded_pay_2, hot_encoded_pay_3, hot_encoded_pay_4, hot_encoded_pay_5, hot_encoded_pay_6]:
    dataframe = pandas.concat([dataframe, i], axis = 1);

print("Data after cleaning: ", dataframe.shape);
print(dataframe.columns.tolist());

Data after cleaning:  (29655, 77)
['EDUCATION', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'LABEL', 'education_1', 'education_2', 'education_3', 'education_4', 'pay_0_-2', 'pay_0_-1', 'pay_0_0', 'pay_0_1', 'pay_0_2', 'pay_0_3', 'pay_0_4', 'pay_0_5', 'pay_0_6', 'pay_0_7', 'pay_0_8', 'pay_2_-2', 'pay_2_-1', 'pay_2_0', 'pay_2_1', 'pay_2_2', 'pay_2_3', 'pay_2_4', 'pay_2_5', 'pay_2_6', 'pay_2_7', 'pay_2_8', 'pay_3_-2', 'pay_3_-1', 'pay_3_0', 'pay_3_1', 'pay_3_2', 'pay_3_3', 'pay_3_4', 'pay_3_5', 'pay_3_6', 'pay_3_7', 'pay_3_8', 'pay_4_-2', 'pay_4_-1', 'pay_4_0', 'pay_4_1', 'pay_4_2', 'pay_4_3', 'pay_4_4', 'pay_4_5', 'pay_4_6', 'pay_4_7', 'pay_4_8', 'pay_5_-2', 'pay_5_-1', 'pay_5_0', 'pay_5_2', 'pay_5_3', 'pay_5_4', 'pay_5_5', 'pay_5_6', 'pay_5_7', 'pay_5_8', 'pay_6_-2', 'pay_6_-1', 'pay_6_0', 'pay_6_2', 'pay_6_3', 'pay_6_4', 'pay_6_5', 'pay_6_6', 'pay_6_7', 'pay_6_8']


In [35]:
# Define label data
label = dataframe['LABEL'];

# Drop ID, SEX, EDUCATION, MARRIAGE, and LABEL from dataframe for features
features = dataframe.drop(columns=["EDUCATION", 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', "LABEL"]);

# Typecasting all values within to int64 because ANN need to be standardized

for i in dataframe.columns:
    dataframe[i] = dataframe[i].astype(int);

# Split the data into training, validation, and testing sets
feature_train, feature_test, label_train, label_test = train_test_split(features, label, test_size = 0.6, random_state = 42);
feature_validation, feature_test, label_validation, label_test = train_test_split(feature_test, label_test, test_size=0.5, random_state = 20);

# Scale the features
scaler = StandardScaler();
feature_train_scaled = scaler.fit_transform(feature_train);
feature_validation_scaled = scaler.fit_transform(feature_validation);
feature_test_scaled = scaler.transform(feature_test);

## Fine Tunning

This code based on Tensorflow Artifical Neural Network.

In [36]:
class ANNModel:

    optimizer = "adam";

    def input_layer(self):
        return keras.layers.Dense(32, input_dim = 69, activation = "sigmoid"); # 11 is total features dimension

    def hidden_layer(self):
        return [
            keras.layers.Dense(1024, activation = "relu", kernel_regularizer=l2(1e-3)),
            keras.layers.Dense(512, activation = "relu", kernel_regularizer=l2(1e-2)),
            keras.layers.Dense(256, activation = "relu", kernel_regularizer=l2(1e-1)),
        ];

    def shallow_hidden_layer(self):
        return [
            keras.layers.Dense(64, activation = "relu"),
            keras.layers.Dense(32, activation = "relu"),
            keras.layers.Dense(32, activation = "relu"),
        ];

    def output_layer(self):
        return keras.layers.Dense(1, activation = "sigmoid"); # Since output is only 1 label.

    def fit(self):
        model = Sequential();
        model.add(self.input_layer());

        for i in [self.hidden_layer(), self.shallow_hidden_layer()]:
            for j in i:
                model.add(j);

        model.add(self.output_layer());

        plot_model(model, to_file="model_architecture.png", show_shapes=True)

        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]);

        model_plot = model.fit(feature_train_scaled, label_train, epochs = 200, validation_data=(feature_validation_scaled, label_validation));

        plt.plot(model_plot.history["loss"], label = "Model Training Loss");
        plt.plot(model_plot.history["val_loss"], label = "Model Validation Loss");
        plt.title("Training and Validation Loss");
        plt.xlabel("Epochs");
        plt.ylabel("Loss");
        plt.legend();
        plt.show();

        return model;

dojo = ANNModel();
model = dojo.fit();

loss, accuracy = model.evaluate(feature_test_scaled, label_test);
print("Loss: ", loss, " Accuracy: ", accuracy);

Epoch 1/200


371/371 [==============================] - 9s 18ms/step - loss: 3.1637 - accuracy: 0.7951 - val_loss: 0.4724 - val_accuracy: 0.8150
Epoch 2/200
371/371 [==============================] - 6s 17ms/step - loss: 0.4621 - accuracy: 0.8173 - val_loss: 0.4563 - val_accuracy: 0.8183
Epoch 3/200
371/371 [==============================] - 6s 17ms/step - loss: 0.4553 - accuracy: 0.8208 - val_loss: 0.4524 - val_accuracy: 0.8199
Epoch 4/200
371/371 [==============================] - 6s 17ms/step - loss: 0.4521 - accuracy: 0.8211 - val_loss: 0.4525 - val_accuracy: 0.8203
Epoch 5/200
371/371 [==============================] - 6s 17ms/step - loss: 0.4504 - accuracy: 0.8210 - val_loss: 0.4513 - val_accuracy: 0.8206
Epoch 6/200
371/371 [==============================] - 6s 17ms/step - loss: 0.4481 - accuracy: 0.8230 - val_loss: 0.4549 - val_accuracy: 0.8205
Epoch 7/200
371/371 [==============================] - 7s 18ms/step - loss: 0.4496 - accuracy: 0.8220 - val_loss: 0.4508 - val_accuracy: 0.8186
Epoc